# Flock Filter

## Drawbacks of UKF

$$
\def\M#1{{[\mathbf{#1}]}}
\def\MM#1#2{{[\mathbf{#1}{#2}]}}
\def\E{\operatorname{E}}
\def\cov{\operatorname{cov}}
\def\T{^\mathsf{T}}
$$

The [unscented Kalman filter](UnscentedKalman.ipynb) is an attempt to preserve non-Gaussian covariances through non-linear transformations, but if there are frequent measurements, then the covariance gets Gaussianized (by way of the unscented transform) anyway, very often. The unscented transform builds its sigma points based on an assumption of Gaussian distribtion, pushes the sigma points through the nonlinear function to get a non-Gaussian point cloud, but then immediately calculates a Gaussian approximation (in the form of a mean and covariance) *and throws away the sigma points*. Each time, the sigma points are constructed anew. For one, this seems wasteful, since you have to do a Cholesky decomposition of the covariance matrix at each measurement. For another, it means that the problem is re-Gaussianized at each measurement. 

The case I am thinking of is the hyperbolic orbit, right near periapse.

In [ ]:
%%capture
%run UnscentedKalman.ipynb

In [ ]:
#Dynamics
rx,ry,vx,vy=sympy.symbols('r_x r_y v_x v_y')
#It's OK to introduce intermediate symbols, since they are still defined in terms
#of the symbols we will be using in the state vector
r=sympy.sqrt(rx**2+ry**2) 
Frx=vx
Fry=vy
Fvx=-rx/r**3
Fvy=-ry/r**3
(F,_)=fjac([rx,ry,vx,vy],[],[Frx,Fry,Fvx,Fvy]) #Don't care about physics matrix

#Measurement equations
mx=2
my=0
rho=sympy.sqrt((rx-mx)**2+(ry-my)**2)
theta=sympy.atan2(ry-my,-(rx-mx)) #Put the branch cut on the x-axis, so that the test orbit doesn't cross it
(g,_)=fjac([rx,ry,vx,vy],[],[rho,theta]) #Don't care about measurement matrix

In [ ]:
n_cloud=1000 #Number of elements in the point cloud
t0=0
t1=20
dt=0.1
ts=np.arange(t0,t1,dt) #Time series
n_t=ts.size

Q = np.diag([0.0, 0.0, 0.0, 0.0])  # Non-physical process noise covariance
R = np.diag([0.1 ** 2,0.01**2])  # Measurement noise covariance

x0=np.array([[8],[2],[-0.5],[0]])
P0=np.diag([0.01,0.01,1e-5,1e-5])

#Monte Carlo initial conditions
nr.seed(3217)
xm = pnoise(n_cloud, P0, x0)

#EKF initial conditions
xe=x0*1
Pe=P0*1

#UKF initial conditions
(xu, W) = sigma(x0, P0)

plt.figure("Non-Gaussian point cloud")
plt.plot([0], [0], 'kx')
dt=ts[1]-ts[0]
for i, t in enumerate(ts):
    #Do all of the plotting
    if i % 20 == 0 or i+1==len(ts):
        if i+1==len(ts) or i==120:
            plt.plot(xm[0, :], xm[1, :], 'r+',label="Monte Carlo point cloud")
            plt.plot(xtm[i,0,:],xtm[i,1,:],'y+',label="Kepler point cloud")
        (xmbar, Pm) = pcalc(xm)
        (cm, sm) = pell(3, Pm[0:2,0:2], xmbar[0:2])
        plt.plot(cm, sm, 'r-',label='Monte Carlo' if i==0 else None)

        if i+1==len(ts):
            plt.plot(xe[0, :], xe[1, :], 'bo',label="EKF estimate")
        (ce, se) = pell(3, Pe[0:2,0:2], xe[0:2])
        plt.plot(ce, se, 'b-',label='Linearized' if i==0 else None)

        if i+1==len(ts):
            plt.plot(xu[0, :], xu[1, :], 'g+',label="Sigma point cloud")
        (xubar, Pu) = sigmainv(xu, W)
        (cu, su) = pell(3, Pu[0:2,0:2], xubar[0:2])
        plt.plot(cu, su, 'g-',label='Unscented' if i==0 else None)
    #Actually step the integrators
    xm = rk4(Fnl, dt, xm, [], nstep=nstep,t0=t)
    xu = rk4(Fnl, dt, xu, [], nstep=nstep,t0=t)
    (xe, Pe) = ekf_timeup(xh_im1=xe, P_im1=Pe, t=t, dt=dt, F=Fnl, Phi=Phinl, Fpar=[],nstep=nstep)
plt.axis('equal')
plt.xlim([-1,1])
plt.ylim([-1.3,1.3])
plt.legend()
plt.show()

Note that in this case, the point cloud isn't really that well-described by any ellipse. Sure, the Monte Carlo and Unscented ellipses cover the covariance, but the covariance is only really enough information if the point cloud is Gaussian. Imagine a long unscented transform from the initial condition, where the distribution really is Gaussian. If we take a measurement right at periapse, the point cloud will have its mean and covariance computed, Kalman gain applied, and from then forward, the distribution will be re-Gaussianized. Imagine that the measurement didn't help much, and that the covariance after is the same as the one before. In this case, the next point cloud will be Gaussian-distributed over the green ellipse. This will put some points *much* closer to the origin, which will fling them pretty far afield from the center.

This drawback applies equally to the Monte Carlo filter, with it's near-infinite point cloud, and the unscented filter, with its finite set of sigma points.

## Flock Filter

This is designed for *either* the Monte Carlo *or* the unscented point clouds. Instead of jumping back and forth between problem domains, we will do our best to keep it in one domain -- that of the point clouds. Especially in our case, the transformation from covariance to point cloud requires a Cholesky decomposition, which among other problems, I don't have code for that on my bare-metal embedded system. So what we will do is:

1. Transform the input mean and covariance into a point cloud *once*, and keep track of the weight $W_j$. For sigma points, use ``sigma()``. For Monte Carlo, use ``perror()``, and each weight is $W_j=\frac{1}{1-N}$ where $N$ is the number of points in the cloud.
2. For each measurement:
   * Propagate the points using the nonlinear transform to the time of the measurement: $$\vec{x}^-_j=\vec{f}(\vec{x}_{j,i-1},\vec{k},t)$$
   * Figure $\M \Gamma$  from the point cloud, just like in the unscented transform:$$
   \begin{align}
   \vec{z}_j=&\vec{g}(\vec{x}^-_j,\vec{k},t) \\
   \hat{\vec{z}}=&\sum_{j}W_j \hat{\vec{z}}_j \\
\M{\Gamma}=&\M{R}+\sum_{j}W_j \left\{\hat{\vec{z}}-\hat{\vec{z}}_j\right\}\left\{\hat{\vec{z}}-\hat{\vec{z}}_j\right\}\T\end{align}$$
   
   * Figure $\M S$ from the point cloud measurement residuals, just like in the unscented transform:
   $$\M{S}=\sum_{j}W_j \left\{\vec{x}^--\vec{x}^-_j\right\}\left\{\hat{z}-\hat{z}_j\right\}\T$$
   * Figure $\M K=\M S\M \Gamma^{-1}$ just like everywhere else
   * For *each point in the cloud*:
     * calculate the correction by multiplying $\M K$ by the measurement residual *for that point*: 
     $$\begin{align}
     \vec{y}_j=&\vec{z}-\vec{z}_j \\
     \vec{x}_j=&\vec{x}^-_j+\M{K}\vec{y}_j
     \end{align}$$

The idea is that we repeatedly go from the point cloud domain to the covariance domain, but only to calculate the Kalman gain. We then use that gain in the point cloud domain. The covariances are made fresh from the cloud at each time it is needed, but the cloud is never again made from the covariance.

We call it a *flock* filter because while the natural disposition of a cloud is to disperse, the measurement corrections applied via $\M K$ cause each point to converge on the correct answer, like a flock of birds will converge on their formation because each bird corrects itself.

The idea that makes this work is that the derivation of $\M K$ does not depend on the actual state or measurement, so it must be able to guide corrections to points anywhere in a cloud.

In [ ]:
def flock_measup(x_mj, W, z, g,gpar, R, t=None):
    """
    :param x_mj  : Time-updated point cloud (at current time but before taking into account current measurement)
    :param W     : Weights of sigma points
    :param z     : current measurement vector
    :param g     : Measurement function
    :param gpar  : Extra parameters to pass to measurement function
    :param R     : Measurement noise covariance matrix, square m x m where m is number of elements of measurement vector
    :return      : A tuple:
        x - Measurement-update point cloud
    """
    def _inv(x):
        """
        Matrix inverse which also handles scalars properly
        :param x:
        :return:
        """
        try:
            from scipy.linalg import inv
            return inv(x)
        except ValueError:
            return np.array([[1.0/x]])
    # Figure Gamma matrix from point cloud 
    z_j=g(x_mj,gpar,t)
    (zh,Gamma)=sigmainv(z_j,W)
    Gamma+=R
    # Figure S matrix from point cloud measurement residuals
    (xh_m,_)=sigmainv(x_mj,W)
    dx=(xh_m-x_mj[:,0].reshape(-1,1))
    dz=(zh  -z_j [:,0].reshape(-1,1))
    dxdz=dx @ dz.T
    S=W[0]*dxdz
    for j in range(1,W.size):
        S+=W[j]*(xh_m-x_mj[:,j].reshape(-1,1))@((zh-z_j[:,j].reshape(-1,1)).T)
    # 5. Calculate the Kalman gain
    K=S @ _inv(Gamma)
    x_j=x_mj.copy()
    for j in range(W.size):
        # 6. Update each point in the cloud
        y_j=z-z_j[:,j].reshape(-1,1)
        x_j[:,j]=(x_mj[:,j].reshape(-1,1)+K@y_j).ravel()
    return x_j

Note that there isn't any way to deal with $\M Q$ this way -- it needs to be dealt with during the time update. With a linear or extended Kalman filter, process noise is added by just adding it to the time-updated state covariance. With an un-augmented unscented filter, we add the process noise to the state covariance that we recover from the sigma points. With the augmented unscented filter, the extra sigma points related to process noise are used, except this only works for the first step.

There are a couple of ways to deal with this:

0. Don't use process noise. This is fine for problems like orbital mechanics where you really do know all of the dynamics, but useless for things like the rocketometer problem.
1. For augmented sigma points, I think what you have to do is to keep track of which points in the point cloud are there because of process noise. Before each time propagation step, add perturbations to the augmented points, based on the current $\M Q$. If $\M Q$ is constant, then you can just calculate these perturbations once, and add them each time. The justification for adding this each step is that this *is* what you get if you repeatedly time-update a linear or EKF filter without measurement updates. The measurement update will correct the points back inward towards the correct value.
2. For unaugmented sigma points or Monte Carlo point clouds, create a random perturbation vector for each point, where the perturbations have zero mean and a covariance of $\M Q$. I think that if $\M Q$ is constant, you can create one random perturbation first, and add it at each step.

In [ ]:
def flock_timeup(x_im1, t_im1, dt, F, Fpar=None, Qperturb=None, odesolve=rk4, nstep=10):
    """
    Do the flock filter time update. Given a previous state estimate in the form of a point cloud, 
    and everything else needed to do the physics, calculate the updated state estimate point cloud
    after taking a time step.
    :param x_im1 : Previous point cloud, stack of N n-element vectors, or n x N matrix
    :param t_im1 : initial time (time of previous measurement)
    :param dt    : time between last and current measurement
    :param F     : Physics function
    :param Fpar  : Extra parameters to pass to physics functions
    :param Qperturb: Process noise perturbation vectors. Stack of L n-element vectors, or n x L matrix.
                     Will be used to perturb last last L points in the cloud. If L==N, then all
                     vectors will be perturbed (method 2 above). If L<N, assume the last L vectors
                     are the augmented sigma points (method 1 above). If not passed, don't use process noise
                     (method 0 above).
    :return      : A tuple:
        xh_m - Time-updated point cloud
    """
    if dt!=0:
        xh_m=odesolve(F,dt,x_im1,k=Fpar,nstep=nstep,t0=t_im1)
    else:
        xh_m=x_im1.copy()
    if Qperturb is not None:
        L=Qperturb.shape[1]
        xh_m[:,-L:]+=Qperturb
    return xh_m

In [ ]:
def flock_step(x_im1, W, z, dt, F, Fpar, g, gpar, Qperturb, R, odesolve=rk4, nstep=10, t=None):
    """
    Call both the EKF time update and measurement update functions to process one measurement.
    :param xh_im1: Previous estimate of the state vector
    :param P_im1 : Previous estimate covariance
    :param z_i   : current measurement vector
    :param dt    : time between last and current measurement
    :param F     : Physics function
    :param Fpar  : Extra parameters to pass to physics function
    :param g     : Measurement function
    :param gpar  : Extra parameters to pass to measurement function
    :param Q     : Process noise covariance matrix, square n x n where n is number of elements of state vector
    :param R     : Measurement noise covariance matrix, square m x m where m is number of elements of measurement vector
    :param odesolve: ODE solver to use, passed along to time update
    :param nstep : Number of substeps to use in the step, passed along to time update
    :return      : A tuple:
        xh_i - New estimated state vector
        P_i  - New estimate covariance matrix
    """
    x_mj=flock_timeup(x_im1=x_im1, t_im1=t-dt, dt=dt, F=F, Fpar=Fpar, Qperturb=Qperturb)
    return flock_measup(x_mj=x_mj, W=W, z=z, g=g, gpar=gpar, R=R, t=t)


In [ ]:
def flock_loop2(x_0,W, t,z,F,Fpar,g,gpar,Q,R):
    N=x_0.shape[1]
    if Q is not None:
        Qperturb=pnoise(N,Q)
    else:
        Qperturb=None
    #Indexes will be [time,vector component,point cloud index]
    x_j=np.zeros((t.size,)+x_0.shape)
    x_j[0,:,:]=x_0
    xh=np.zeros((x_0.shape[0],t.size))
    P=np.zeros((t.size,x_0.shape[0],x_0.shape[0]))
    xh_i,P_i=sigmainv(x_0,W)
    xh[:,0]=xh_i.ravel()
    P[0,:,:]=P_i
    for i in range(1,ts.size):
        x_j[i,:,:]=flock_step(x_im1=x_j[i-1,:,:],W=W,
                              z=z[:,i].reshape(-1,1),
                              dt=t[i]-t[i-1],t=t[i],
                              F=F,Fpar=Fpar,g=g,gpar=gpar,
                              Qperturb=Qperturb,R=R)
        xh_i,P_i=sigmainv(x_j[i,:,:],W)
        xh[:,i]=xh_i.ravel()
        P[i,:,:]=P_i
    return xh,P,x_j

In [ ]:
def trueState(t,xt0):
    """
    Calculate the true trajectory, given the true initial condition and the physics function F
    """
    return kepler(xt0,None,t)

xt=trueState(t=ts,xt0=x0)

plt.figure("Flock True trajectory")
plt.plot(xt[0,:],xt[1,:],label='true trajectory')
plt.plot(0,0,'k+',label='center')
plt.plot(mx,my,'r+',label='station')
plt.legend()
plt.axis('equal')
plt.show()

In [ ]:
def trueObs(xt,g,R=None,seed=3218):
    #Size the problem
    s=xt.shape
    n=s[0]
    xt0=xt[:,0].reshape(-1,1)
    z=np.zeros((g(xt0,None,None).size,s[1]))
    #Clean observations
    for i in range(s[1]):
        z[:,i]=g(xt[:,i].reshape(n,1),None,None).ravel()
  
    if R is not None:
        if seed is not None:
            #We want random noise, but the *same* random noise each time we run
            nr.randn(seed)
        pn=pnoise(s[1],R) #Measurement noise
        z=z+pn            #Noisy Observations
    return z

zt=trueObs(xt,gnl)
zn=trueObs(xt,gnl,R)
plt.figure("Flock range Observations")
plt.plot(ts,zt[0,:],'-')
plt.plot(ts,zn[0,:],'.')
plt.figure("Flock azimuth Observations")
plt.plot(ts,zt[1,:],'-')
plt.plot(ts,zn[1,:],'.')
plt.show()

In [ ]:
#Initial condition error
dx0=np.array([[0.1],[0.1],[0.1],[0.1]])

#Point cloud
nr.rand(3217)
xm=pnoise(n_cloud, P0, x0+dx0)
Wm=np.zeros(n_cloud,)+1/(n_cloud-1)
nr.randn(3219)
#Qperturbm=pnoise(n_cloud,Q)
#Unaugmented sigma points
(xu,Wu)=sigma(x0+dx0,P0)
nr.randn(3220)
#Qperturbu=pnoise(xu.shape[1],Q)

%time (xh_m,P_m,x_m)=flock_loop2(xm, Wm,ts,zn,F,[],g,[],Q=None,R=R)
print(xh_m.shape,P_m.shape,x_m.shape)
%time (xh_u,P_u,x_u)=flock_loop2(xu, Wu,ts,zn,F,[],g,[],Q=None,R=R)
print(xh_u.shape,P_u.shape,x_u.shape)

In [ ]:
plt.figure("Flock MC solution")
plt.plot(0,0,'k+')
plt.plot(xt  [0,:],xt  [1,:],'r+-',label='True')
plt.plot(xh_m[0,:],xh_m[1,:],'g+',label='MC mean')
plt.plot(xh_u[0,:],xh_u[1,:],'b+',label='sigma mean')
for i in range(0,ts.size):
    (cm, sm) = pell(3, P_m[i,0:2,0:2], xh_m[0:2,i].reshape(-1,1))
    plt.plot(cm, sm, 'g-',label='MC cov' if i==0 else None)
    (cu, su) = pell(3, P_u[i,0:2,0:2], xh_u[0:2,i].reshape(-1,1))
    plt.plot(cu, su, 'b-',label='Sigma cov' if i==0 else None)
    if i<3:
        plt.plot(x_m[i,0,:],x_m[i,1,:],'g*',label='MC points' if i==0 else None)
        plt.plot(x_u[i,0,:],x_u[i,1,:],'b*',label='sigma points' if i==0 else None)
plt.legend()
plt.axis('equal')

zh_m=trueObs(xh_m,g)
zh_u=trueObs(xh_u,g)

plt.figure("Flock range measurements")
plt.plot(ts, zt[0,:], 'r-',label='Actual')
plt.plot(ts, zh_m[0,:], 'g+',label='MC');
plt.plot(ts, zh_u[0,:], 'b+',label='Sigma');
plt.legend()

plt.figure("Flock range residuals")
plt.plot(ts, zt[0,:]-zh_m[0,:],'g+', label='MC residuals')
plt.plot(ts, zt[0,:]-zh_u[0,:],'b+', label='Sigma residuals')
plt.plot(ts, zt[0,:]-zn  [0,:],'r+', label='measurement noise')
plt.plot(ts, ts*0, 'k-',label='residual=0');
plt.legend()

plt.figure("Flock azimuth measurements")
plt.plot(ts, zt  [1,:], 'r-',label='Actual')
plt.plot(ts, zh_m[1,:], 'g+',label='MC');
plt.plot(ts, zh_u[1,:], 'b+',label='Sigma');
plt.legend()

plt.figure("Flock azimuth residuals")
plt.plot(ts, zt[1,:]-zh_m[1,:],'g+', label='MC residuals')
plt.plot(ts, zt[1,:]-zh_u[1,:],'b+', label='Sigma residuals')
plt.plot(ts, zt[1,:]-zn  [1,:],'r+', label='measurement noise')
plt.plot(ts, ts*0, 'k-',label='residual=0');
plt.legend()

plt.show()